In [ ]:
from pathlib import Path
from datetime import datetime

from aust_covid.inputs import load_calibration_targets, load_who_data, load_serosurvey_data

PROJECT_PATH = Path().resolve().parent

In [ ]:
class TexDoc:
    def __init__(self, path, doc_name):
        self.content = []
        self.packages = []
        self.path = path
        self.doc_name = f'{doc_name}.tex'
    
    def add_line(self, line):
        self.content.append(line)
    
    def write_line(self, line):
        self.file.write(f'{line}\n')

    def start_doc(self, bib_filename, title):
        self.write_line('\\documentclass{article}')
        for line in self.packages:
            self.write_line(line)
        self.write_line(f'\\addbibresource{{{bib_filename}.bib}}')
        self.write_line(f'\\title{{{title}}}')
        self.write_line('\\begin{document}')

    def populate_doc(self, elements):
        for element in elements:
            self.file.write(element)
        self.file.write('\n')
        
    def finish_doc(self):
        self.write_line('\\printbibliography')
        self.write_line('\\end{document}')
        
    def write_doc(self):
        with open(self.path / 'supplement' /self.doc_name, 'w') as self.file:
            self.start_doc('austcovid', 'Supplemental Appendix')
            self.populate_doc(self.content)
            self.finish_doc()
            
class StandardTexDoc(TexDoc):
    def __init__(self, path, doc_name):
        super().__init__(path, doc_name)
        core_packages = [
            '\\usepackage{biblatex}',
            '\\usepackage{hyperref}',
        ]    
        self.packages += core_packages

In [ ]:
app_doc = StandardTexDoc(PROJECT_PATH, 'supplement')

In [ ]:
targets_rolling_window = 7
smoothed_composite_cases = load_calibration_targets(datetime(2021, 12, 15), targets_rolling_window, app_doc)

In [ ]:
death_data, death_targets_text = load_who_data(targets_rolling_window)
death_targets_text

In [ ]:
app_doc.add_line(death_targets_text)
app_doc.write_doc()